<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BERTimbau_large_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [ ]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 6.9 MB/s 
     |████████████████████████████████| 432 kB 68.0 MB/s 
     |████████████████████████████████| 163 kB 74.7 MB/s 
     |████████████████████████████████| 7.6 MB 57.8 MB/s 
     |████████████████████████████████| 115 kB 61.9 MB/s 
     |████████████████████████████████| 212 kB 52.9 MB/s 
     |████████████████████████████████| 127 kB 73.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [ ]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
!awk NF < dump.txt > kant.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  4818k      0 --:--:-- --:--:-- --:--:-- 4818k


In [ ]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-large-portuguese-cased')

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!shuf -n1000 kant.txt > kant_teste.txt

In [ ]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-7f450cf4e9c4aacd/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

exemplo de entrada

In [ ]:
tokenized_datasets["train"][4]

{'input_ids': [101, 1627, 391, 16581, 340, 125, 20229, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 32


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],    
)

In [ ]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14821
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9270


Step,Training Loss
500,2.462100
1000,2.323800
1500,2.217700
2000,2.118600
2500,2.082300
3000,2.059400
3500,1.993600
4000,1.911600
4500,1.875000
5000,1.821600


Step,Training Loss
500,2.462100
1000,2.323800
1500,2.217700
2000,2.118600
2500,2.082300
3000,2.059400
3500,1.993600
4000,1.911600
4500,1.875000
5000,1.821600




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 51min 6s, sys: 22min 14s, total: 1h 13min 20s
Wall time: 1h 13min 19s


TrainOutput(global_step=9270, training_loss=1.8957204709675448, metrics={'train_runtime': 4399.4261, 'train_samples_per_second': 33.688, 'train_steps_per_second': 2.107, 'total_flos': 8633444480613120.0, 'train_loss': 1.8957204709675448, 'epoch': 10.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1958
  Batch size = 8


Perplexity: 5.23


In [ ]:
eval_results

{'eval_loss': 1.8296008110046387,
 'eval_runtime': 4.6816,
 'eval_samples_per_second': 416.31,
 'eval_steps_per_second': 52.119,
 'epoch': 10.0}

In [ ]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [ ]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config BertConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29794
}

loading configuration file ./KantaiBERT/config.json
Model config BertConfig {
  "_na

In [ ]:
fill_mask("O rapaz olhou para o [MASK] ")

[{'score': 0.2498740553855896,
  'token': 8105,
  'token_str': 'chão',
  'sequence': 'O rapaz olhou para o chão'},
 {'score': 0.14443957805633545,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'O rapaz olhou para o céu'},
 {'score': 0.060382332652807236,
  'token': 16909,
  'token_str': 'muro',
  'sequence': 'O rapaz olhou para o muro'},
 {'score': 0.05292540788650513,
  'token': 9778,
  'token_str': 'menino',
  'sequence': 'O rapaz olhou para o menino'},
 {'score': 0.03763076663017273,
  'token': 528,
  'token_str': 'mar',
  'sequence': 'O rapaz olhou para o mar'}]

In [ ]:
fill_mask("A moça olhou para o [MASK] ")

[{'score': 0.3073830306529999,
  'token': 8105,
  'token_str': 'chão',
  'sequence': 'A moça olhou para o chão'},
 {'score': 0.14119493961334229,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'A moça olhou para o céu'},
 {'score': 0.0516817681491375,
  'token': 9778,
  'token_str': 'menino',
  'sequence': 'A moça olhou para o menino'},
 {'score': 0.04971173405647278,
  'token': 528,
  'token_str': 'mar',
  'sequence': 'A moça olhou para o mar'},
 {'score': 0.034284040331840515,
  'token': 16909,
  'token_str': 'muro',
  'sequence': 'A moça olhou para o muro'}]

In [ ]:
fill_mask("Comprou uma [MASK] na loja. ")

[{'score': 0.2589082717895508,
  'token': 7924,
  'token_str': 'camisa',
  'sequence': 'Comprou uma camisa na loja.'},
 {'score': 0.09350335597991943,
  'token': 3743,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.07137694209814072,
  'token': 9104,
  'token_str': 'cadeira',
  'sequence': 'Comprou uma cadeira na loja.'},
 {'score': 0.06464625149965286,
  'token': 5896,
  'token_str': 'muda',
  'sequence': 'Comprou uma muda na loja.'},
 {'score': 0.044911906123161316,
  'token': 12252,
  'token_str': 'flor',
  'sequence': 'Comprou uma flor na loja.'}]

In [ ]:
fill_mask("A mulher não é [MASK]. ")

[{'score': 0.1710393875837326,
  'token': 1766,
  'token_str': 'má',
  'sequence': 'A mulher não é má.'},
 {'score': 0.062111109495162964,
  'token': 14578,
  'token_str': 'fraca',
  'sequence': 'A mulher não é fraca.'},
 {'score': 0.057788118720054626,
  'token': 19462,
  'token_str': 'santa',
  'sequence': 'A mulher não é santa.'},
 {'score': 0.05032818764448166,
  'token': 19543,
  'token_str': 'bonita',
  'sequence': 'A mulher não é bonita.'},
 {'score': 0.039439089596271515,
  'token': 2745,
  'token_str': 'tudo',
  'sequence': 'A mulher não é tudo.'}]

In [ ]:
fill_mask("O homem não é [MASK]. ")

[{'score': 0.10878247022628784,
  'token': 13380,
  'token_str': 'perfeito',
  'sequence': 'O homem não é perfeito.'},
 {'score': 0.1042620912194252,
  'token': 11775,
  'token_str': 'mau',
  'sequence': 'O homem não é mau.'},
 {'score': 0.057244572788476944,
  'token': 7255,
  'token_str': 'religioso',
  'sequence': 'O homem não é religioso.'},
 {'score': 0.052857719361782074,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.03668659180402756,
  'token': 7148,
  'token_str': 'padre',
  'sequence': 'O homem não é padre.'}]

In [ ]:
fill_mask("Ele é um bom [MASK]. ")

[{'score': 0.2591373324394226,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.11883705854415894,
  'token': 13254,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'},
 {'score': 0.061398785561323166,
  'token': 4170,
  'token_str': 'marido',
  'sequence': 'Ele é um bom marido.'},
 {'score': 0.04507303610444069,
  'token': 1568,
  'token_str': 'pai',
  'sequence': 'Ele é um bom pai.'},
 {'score': 0.04307873547077179,
  'token': 1417,
  'token_str': 'filho',
  'sequence': 'Ele é um bom filho.'}]

In [ ]:
fill_mask("Ela é uma boa [MASK]. ")

[{'score': 0.36007604002952576,
  'token': 15799,
  'token_str': 'criatura',
  'sequence': 'Ela é uma boa criatura.'},
 {'score': 0.28757745027542114,
  'token': 2760,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.05713978782296181,
  'token': 8932,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.03646063804626465,
  'token': 2196,
  'token_str': 'mãe',
  'sequence': 'Ela é uma boa mãe.'},
 {'score': 0.026936767622828484,
  'token': 8410,
  'token_str': 'alma',
  'sequence': 'Ela é uma boa alma.'}]

In [ ]:
fill_mask("Faz de conta que ainda é [MASK]. ")

[{'score': 0.24292521178722382,
  'token': 3361,
  'token_str': 'vivo',
  'sequence': 'Faz de conta que ainda é vivo.'},
 {'score': 0.20824936032295227,
  'token': 4964,
  'token_str': 'criança',
  'sequence': 'Faz de conta que ainda é criança.'},
 {'score': 0.057308949530124664,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Faz de conta que ainda é homem.'},
 {'score': 0.041546210646629333,
  'token': 7148,
  'token_str': 'padre',
  'sequence': 'Faz de conta que ainda é padre.'},
 {'score': 0.030354171991348267,
  'token': 19543,
  'token_str': 'bonita',
  'sequence': 'Faz de conta que ainda é bonita.'}]

In [ ]:
fill_mask("Depois da tempestade vem a [MASK]. ")

[{'score': 0.09127185493707657,
  'token': 9856,
  'token_str': 'chuva',
  'sequence': 'Depois da tempestade vem a chuva.'},
 {'score': 0.08858317881822586,
  'token': 8611,
  'token_str': 'esperança',
  'sequence': 'Depois da tempestade vem a esperança.'},
 {'score': 0.07901965081691742,
  'token': 14161,
  'token_str': 'alegria',
  'sequence': 'Depois da tempestade vem a alegria.'},
 {'score': 0.06509854644536972,
  'token': 15685,
  'token_str': 'felicidade',
  'sequence': 'Depois da tempestade vem a felicidade.'},
 {'score': 0.06086253002285957,
  'token': 2434,
  'token_str': 'vitória',
  'sequence': 'Depois da tempestade vem a vitória.'}]

In [ ]:
fill_mask("Mais vale um pássaro na mão do que [MASK] voando. ")

[{'score': 0.18497487902641296,
  'token': 1342,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.08733828365802765,
  'token': 17804,
  'token_str': 'pássaros',
  'sequence': 'Mais vale um pássaro na mão do que pássaros voando.'},
 {'score': 0.07314687967300415,
  'token': 222,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.055554311722517014,
  'token': 368,
  'token_str': 'ele',
  'sequence': 'Mais vale um pássaro na mão do que ele voando.'},
 {'score': 0.05531582981348038,
  'token': 682,
  'token_str': 'dois',
  'sequence': 'Mais vale um pássaro na mão do que dois voando.'}]

In [ ]:
fill_mask("Não tinha [MASK], mas almoçou mesmo assim. ")

[{'score': 0.13627029955387115,
  'token': 11062,
  'token_str': 'fome',
  'sequence': 'Não tinha fome, mas almoçou mesmo assim.'},
 {'score': 0.12296352535486221,
  'token': 3883,
  'token_str': 'carro',
  'sequence': 'Não tinha carro, mas almoçou mesmo assim.'},
 {'score': 0.04881886765360832,
  'token': 12215,
  'token_str': 'costume',
  'sequence': 'Não tinha costume, mas almoçou mesmo assim.'},
 {'score': 0.028149621561169624,
  'token': 1105,
  'token_str': 'casa',
  'sequence': 'Não tinha casa, mas almoçou mesmo assim.'},
 {'score': 0.0277580376714468,
  'token': 3495,
  'token_str': 'dinheiro',
  'sequence': 'Não tinha dinheiro, mas almoçou mesmo assim.'}]

In [ ]:
fill_mask("Bebeu um copo d'água, pois tinha [MASK]. ")

[{'score': 0.8828673362731934,
  'token': 2496,
  'token_str': 'sede',
  'sequence': "Bebeu um copo d'água, pois tinha sede."},
 {'score': 0.057927437126636505,
  'token': 11062,
  'token_str': 'fome',
  'sequence': "Bebeu um copo d'água, pois tinha fome."},
 {'score': 0.008996760472655296,
  'token': 7672,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."},
 {'score': 0.008743658661842346,
  'token': 6272,
  'token_str': 'vontade',
  'sequence': "Bebeu um copo d'água, pois tinha vontade."},
 {'score': 0.005220021586865187,
  'token': 11334,
  'token_str': 'sono',
  'sequence': "Bebeu um copo d'água, pois tinha sono."}]

In [ ]:
fill_mask("Sem saber, ele descobriu um [MASK]. ")

[{'score': 0.39997851848602295,
  'token': 11021,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.1376154124736786,
  'token': 16269,
  'token_str': 'mistério',
  'sequence': 'Sem saber, ele descobriu um mistério.'},
 {'score': 0.03884365037083626,
  'token': 18761,
  'token_str': 'pecado',
  'sequence': 'Sem saber, ele descobriu um pecado.'},
 {'score': 0.03584948182106018,
  'token': 8332,
  'token_str': 'negócio',
  'sequence': 'Sem saber, ele descobriu um negócio.'},
 {'score': 0.034535638988018036,
  'token': 8019,
  'token_str': 'fenômeno',
  'sequence': 'Sem saber, ele descobriu um fenômeno.'}]

In [ ]:
fill_mask("Pedro fez fortuna vendendo [MASK]. ")

[{'score': 0.15810398757457733,
  'token': 2978,
  'token_str': 'livros',
  'sequence': 'Pedro fez fortuna vendendo livros.'},
 {'score': 0.15282924473285675,
  'token': 5976,
  'token_str': 'escravos',
  'sequence': 'Pedro fez fortuna vendendo escravos.'},
 {'score': 0.0647987425327301,
  'token': 2745,
  'token_str': 'tudo',
  'sequence': 'Pedro fez fortuna vendendo tudo.'},
 {'score': 0.04663674160838127,
  'token': 6349,
  'token_str': 'jornais',
  'sequence': 'Pedro fez fortuna vendendo jornais.'},
 {'score': 0.04531889781355858,
  'token': 20556,
  'token_str': 'diamantes',
  'sequence': 'Pedro fez fortuna vendendo diamantes.'}]

In [ ]:
fill_mask("Sem medo de ser [MASK]. ")

[{'score': 0.7325240969657898,
  'token': 8540,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.06665431708097458,
  'token': 3382,
  'token_str': 'visto',
  'sequence': 'Sem medo de ser visto.'},
 {'score': 0.027373142540454865,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Sem medo de ser homem.'},
 {'score': 0.017596738412976265,
  'token': 1568,
  'token_str': 'pai',
  'sequence': 'Sem medo de ser pai.'},
 {'score': 0.016455546021461487,
  'token': 7148,
  'token_str': 'padre',
  'sequence': 'Sem medo de ser padre.'}]